## Rosstat Graph Crawler: примеры использования<a class="anchor" id="0-bullet">
[1. Поиск по содержимому страниц](#1-bullet)  
[2. Сбор названий таблиц в узлах графа](#2-bullet)  
[3. Поиск таблиц по ключевым словам в названии](#3-bullet)  

In [1]:
!pip install -r requirements.txt --user

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
from src.crawler import *

[nltk_data] Downloading package stopwords to /Users/Yulia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

### 1. Поиск по содержимому веб-страниц<a class="anchor" id="1-bullet"></a>
[назад к оглавлению](#0-bullet)

[Опубликованный](https://data-in.ru/data-catalog/datasets/152/) граф сайта Росстата в поле `document` содержит HTML-код узлов графа. Поиск по данному полю может служить аналогом поиска по сайту. В следующей ячейке необходимо присвоить переменной `text_to_search` строковое значение — искомое слово или словосочетание. После выполнения кода вернется датафрейм с идентификаторами `id` и url-адресами `path` тех узлов графа, где было найдено содержимое `text_to_search`.

Поскольку поиск происходит по всем 104711 узлам графа, выполнение кода может занять некоторое время.

In [5]:
text_to_search = 'среднедушевые денежные доходы'

connector = Graph()
search_result = connector.search(text_to_search)

In [6]:
search_result_df = pd.DataFrame(data=search_result)
search_result_df

,id,path
0,2329,https://gks.ru/free_doc/new_site/vndn-2020/Материалы наблюдения.html
1,4303,https://gks.ru/folder/13397
2,17588,http://www.gks.ru/free_doc/new_site/vndn-2019/Материалы наблюдения.html
3,17603,http://www.gks.ru/free_doc/new_site/vndn-2018/Материалы наблюдения.html
4,20328,https://sml.gks.ru/itogy3
5,73746,https://pskovstat.gks.ru/folder/30733
6,74678,https://krsdstat.gks.ru/standards_of_life_ra
7,76524,https://astrastat.gks.ru/folder/61318
8,85249,https://kostroma.gks.ru/folder/26037
9,104119,http://www.gks.ru/free_doc/new_site/vndn-2017/technicalinformation.html


Для сохранения таблицы `search_result_df` на локальный компьютер запустите ячейку:

In [7]:
search_result_df.to_excel('search_result.xlsx')

### 2. Сбор названий таблиц в узлах графа<a class="anchor" id="2-bullet"></a>
[назад к оглавлению](#0-bullet)

Поиск данных может быть сведен к задаче поиска таблиц с релевантными названиями. Код ниже позволит собрать названия таблиц с тех узлов графа, где информация хранится в форматах `.docx`, `.xlsx`, `.xls`, `.htm` и в архивах `.zip`, `.rar` с вложенными файлами перечисленных расширений.

Чтобы собрать названия таблиц с конкретных узлов графа, необходимо передать в переменную `graph_ids` итерируемый объект, например, список, со значениями `id` узлов. В результате выполнения кода, будут заполнены два датафрейма — `df_success` и `df_failure`.

В `df_success` хранится информация о тех таблицах, названия которых удалось распознать:
- `graph_id` — идентификатор узла графа, где была найдена таблица,
- `path` - url-адрес узла,
- `idx` - порядковый номер таблицы в документе,
- `name` - название таблицы,
- `n_rows` и `n_columns`- число строк и число столбцов в таблице,
- `unit` — предполагаемые единицы измерения данных, если таковые указаны в круглых скобках в конце наименования таблицы,
- `number` — номер таблицы, заданный авторами документа.

Предложенные парсеры не всегда могут определить название. Это может быть связано с ограничениями выдвинутых нами гипотез о структуре документов. К тому же таблица может не иметь названия в принципе, например, когда таблицу используют как инструмент форматирования для размещения в ней иллюстраций в документе Word. В таком случае парсер вернет пустое название таблицы. В датафрейме `df_success` строки с пустыми названиями таблиц опущены.

В датафрейм `df_failure` войдут записи об узлах графа, которые не удалось обработать, а именно идентификатор узла `graph_id`, url-адрес узла `path` и сообщение об ошибке `message`.

В зависимости от конфигураций компьютера, сбор названий таблиц со всего графа может занять несколько часов.

In [8]:
graph_ids = [27949, 27950]

# graph_ids = range(2066, 106777) # все узлы графа

In [9]:
df_success = pd.DataFrame()
df_failure = pd.DataFrame()

In [10]:
for graph_id in tqdm(graph_ids):
    df = crawl_graph(graph_id)
    if df._name == 'df_success':
        df_success = pd.concat([df_success, df], axis=0)
    elif df._name == 'df_failure':
        df_failure = pd.concat([df_failure, df], axis=0)

100%|██████████| 2/2 [00:31<00:00, 15.98s/it]


In [11]:
df_success

,idx,name,n_rows,n_columns,unit,number,graph_id,path
0,0,"Средняя заработная плата педагогических работников образовательных, медицинских организаций или организаций, оказывающих социальные услуги детям-сиротам и детям, оставшимся без попечения родителей* государственной и муниципальной форм собственности за январь - сентябрь 2019 года",15,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
1,0,Средняя заработная плата педагогических работников дошкольных образовательных организаций государственной и муниципальной форм собственности за январь - сентябрь 2019 года,39,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
2,0,Средняя заработная плата педагогических работников образовательных организаций общего образования государственной и муниципальной форм собственности за январь - сентябрь 2019 года,39,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
3,0,Средняя заработная плата педагогических работников организаций дополнительного образования детей* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,39,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
4,0,Средняя заработная плата преподавателей и мастеров производственного обучения образовательных организаций начального и среднего профессионального образования государственной и муниципальной форм собственности за январь - сентябрь 2019 года,24,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
5,0,Средняя заработная плата преподавателей образовательных организаций высшего профессионального образования государственной и муниципальной форм собственности за январь - сентябрь 2019 года,11,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
6,0,Средняя заработная плата работников учреждений культуры государственной и муниципальной форм собственности за январь - сентябрь 2019 года,38,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
7,0,Средняя заработная плата социальных работников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,37,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
8,0,"Средняя заработная плата среднего медицинского (фармацевтического) персонала (персонала, обеспечивающего условия для предоставления медицинских услуг)* государственной и муниципальной форм собственности за январь - сентябрь 2019 года",38,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip
9,0,"Средняя заработная плата врачей и работников медицинских организаций, имеющих высшее медицинское (фармацевтическое) или иное высшее образование, предоставляющих медицинские услуги (обеспечивающих предоставление медицинских услуг)* государственной и муниципальной форм собственности за январь - сентябрь 2019 года",38,13,None,NaN,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и опла

In [12]:
df_failure

,graph_id,path,message
0,27950,https://kostroma.gks.ru/storage/mediabank/w6kNikQM/Средние потребительские цены и тарифы на основные виды социально-значимых услуг по Костромской области в 2020 году.rtf,"подходящий парсер не найден (тип объекта application/rtf, расширение rtf)"


Для сохранения таблиц `df_success` и `df_failure` запустите ячейку:

In [13]:
df_success.to_excel('success.xlsx')
df_failure.to_excel('failure.xlsx')

Чтобы сохранить **файлы** из выбранных узлов на локальный компьютер, необходимо передать идентификаторы узлов в список `graph_ids` ниже и выполнить код в ячейке. Файлы сохранятся в директорию, из которой исполняется данный `.ipynb`; имя файла будет соответствовать `id` узла графа.

In [14]:
graph_ids = [27949, 27950]

for graph_id in tqdm(graph_ids):
    graph_record = Graph().get_graph_record(graph_id)
    GraphNode(graph_record).save_file()

100%|██████████| 2/2 [00:00<00:00,  7.25it/s]


### 3. Поиск таблиц по ключевым словам в названии<a class="anchor" id="3-bullet"></a>
[назад к оглавлению](#0-bullet)

Кодом ниже можно воспользоваться для поиска данных по ключевым словам в названиях таблиц. Под капотом происходит расчет косинусного расстояния между названием таблицы и предложением с ключевыми словами. Чем ближе значение косинусного расстояния к единице, тем более похоже название таблицы на ключевое предложение. 

Предложения с ключевыми словами перечислены в списке `keywords`. Названия таблиц графа должны быть переданы в виде датафрейма с обязательными полями `name` (название таблицы), `graph_id` и `path` (`id` и `path` узла графа). В качестве примера воспользуемся датафреймом `df_success`, полученным в предыдущем пункте. Порог косинусного расстояния нужно задать в параметре `threshold` функции `compare_names`. На выходе мы получим датафрейм с полями:
- `graph_id` — в каком узле графа располагается релевантная таблица, 
- `path` — url-адрес узла,
- `table_name` — наименование таблицы,
- `keyword` — относительно какого предложения рассчитывалось косинусное расстояние,
- `cos_sim` — значение косинусного расстояния. В датафрейм не войдут записи, для которых `cos_sim` < `threshold`.

Обработка может занять существенное время, в зависимости от размера переданного датафрейма с названиями таблиц.

In [15]:
keywords = [
    'Заработная плата',
    'Распространенность курения',
    'Число больничных коек по специализации',
]

df_cos_sim = compare_names(parsed_names_df=df_success, keywords=keywords, threshold=0.4)

In [16]:
df_cos_sim

,graph_id,path,table_name,keyword,cos_sim
0,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата социальных работников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
1,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата научных сотрудников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
2,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата социальных работников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
3,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата научных сотрудников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
4,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата социальных работников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
5,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата научных сотрудников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
6,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата социальных работников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248
7,27949,https://kostroma.gks.ru/storage/mediabank/H8NtbC10/Архив итогов федерального статистического наблюдения о численности и оплате труда отдельных категорий работников.zip,Средняя заработная плата научных сотрудников* государственной и муниципальной форм собственности за январь - сентябрь 2019 года,Заработная плата,0.408248


Для сохранения таблицы `df_cos_sim` запустите ячейку:

In [17]:
df_cos_sim.to_excel('cos_sim.xlsx')

Чтобы сохранить файлы из выбранных узлов на локальный компьютер, необходимо передать идентификаторы узлов в список `graph_ids` ниже и выполнить код в ячейке. Файлы сохранятся в директорию, из которой исполняется данный `.ipynb`; имя файла будет соответствовать номеру узла графа.

In [18]:
graph_ids = [27949, 27950]

for graph_id in tqdm(graph_ids):
    graph_record = Graph().get_graph_record(graph_id)
    GraphNode(graph_record).save_file()

100%|██████████| 2/2 [00:00<00:00, 11.57it/s]
